Hi, Marcel! 

Hope it'll help you to run my code quicker :)

Alex

## Input format

| ID |     Name    | FA_Name | Value |
|---:|:-----------:|:-------:|-------|
|  1 | Revenue_123 | Revenue | -100  |
where **ID** - is actual Business Processes ID, **Name** is the full name of FinancialAccount (not sure that it's required, but initially as I guess one has something like "Revenue_1", "Revenue_2" for different product and only after application of Coarse-graining financial accounts we are getting **FA_Name** value. **Value** is just a amount.

Then on that DataFrame you need to apply *split_credit_debit()* - splits one column **Value** into two columns **Debit** and **Credit**.

After that you should get the following structure:

| ID |     Name    | FA_Name | Value | Debit | Credit | from |
|---:|:-----------:|:-------:|-------|-------|--------|------|
|  1 | Revenue_123 | Revenue | -100  | 0.0   | 1.0    | True |

here **from** is for visualization purposes only (different colors of ingoing/outgoing edges)
## Extraction signatures of BP
For that purpose I've implemented aggregation function over *groupby()* object, as input it uses one BP – a part of original DF with the same ID value – and return a string like
```
'([(\\'Revenue\\', 0.8336558167088283), (\\'Tax\\', 0.1663441832911717)], [(\\'TradeReceivables\\', 1.0)])'
```
then we leave only unique signatures (SAME FA from left and right part AND EXACTLY EQUAL coefficients).
## Construct FSN
Now we can construct FSN from our DataFrame, *build()* method of *FSN* class. 

I recommend to check the construction via *get_FA()* method, it should return the set of Financial Account nodes.
## RandomWalks
#### Already implemented step strategies
1. Pure RandomWalk (uniform probabilities, follows the direction)
2. MetaDef (change directions, uniform probabilities)
3. MetaDefWeighted (change directions, weighted probabilities "rich gets richer")
4. MetaDiff (change directions, probabilities depend on the differences between edges)

For test purpose you can try *randomWalk()* from NetEmbs.FSN.utils, it returns the sampled sequence. 

**BUT**, I've implemented special helper funciton for adding to original DataFrame a column with N top joint-appearance nodes. It's *add_similar()* function from NetEmbs.FSN.utils.

You also can test it with different step strategies: "DefUniform", "DefWeighted", "MetaUniform", "MetaWeighted", "MetaDiff" (version argument).

### Example with sampled data

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
plt.rcParams["figure.figsize"] = [20,10]
from NetEmbs.DataProcessing.normalize import normalize
from NetEmbs.DataProcessing.connect_db import *
from NetEmbs.GenerateData.complex_df import sales_collections
d = normalize(upload_data("Simulation/FSN_Data.db", limit=40))
d.head(15)

,ID,Name,FA_Name,Value,Debit,Credit,from
0,1,Revenue_12,Revenue,-197.000000,0.000000,0.833656,True
1,1,Tax_12,Tax,-39.308553,0.000000,0.166344,True
2,1,TradeReceivables_12,TradeReceivables,236.308553,1.000000,0.000000,False
3,2,TradeReceivables_12,TradeReceivables,-236.308553,0.000000,1.000000,True
4,2,Cash12,Cash,236.308553,1.000000,0.000000,False
5,3,CostOfSales_9,CostOfSales,197.000000,1.000000,0.000000,False
6,3,Inventory_9,Inventory,-197.000000,0.000000,1.000000,True
7,4,PersonnelExpenses_0,PersonnelExpenses,-34.000000,0.000000,0.202381,True
8,4,OtherExpenses_0,OtherExpenses,-36.000000,0.000000,0.214286,True
9,4,PrepaidExpenses_0,PrepaidExpenses,-98.000000,0.000000,0.583333,True


In [12]:
d.groupby("ID").apply(get_signature)

0    Revenue
1        Tax
Name: FA_Name, dtype: object
0    Revenue
1        Tax
Name: FA_Name, dtype: object
3    TradeReceivables
Name: FA_Name, dtype: object
6    Inventory
Name: FA_Name, dtype: object
7    PersonnelExpenses
8        OtherExpenses
9      PrepaidExpenses
Name: FA_Name, dtype: object
11    Revenue
12        Tax
13     kvuacn
Name: FA_Name, dtype: object
18    TradeReceivables
19              cicnuj
20              mgvcyk
21              ehdtdm
Name: FA_Name, dtype: object
24    Inventory
Name: FA_Name, dtype: object
25    PersonnelExpenses
26        OtherExpenses
27      PrepaidExpenses
Name: FA_Name, dtype: object
29    Revenue
30        Tax
Name: FA_Name, dtype: object
32    TradeReceivables
Name: FA_Name, dtype: object
35    Inventory
Name: FA_Name, dtype: object
37    PersonnelExpenses
38        OtherExpenses
39      PrepaidExpenses
Name: FA_Name, dtype: object


""


In [ ]:
from NetEmbs.FSN.graph import FSN
fsn = FSN()
fsn.build(d, name_column="FA_Name")
fsn.nodes()

In [ ]:
fsn.get_BP()

In [ ]:
from NetEmbs.Vis.plots import plotFSN
plotFSN(fsn, edge_labels=False)

In [ ]:
from NetEmbs.FSN.utils import add_similar, get_pairs
add_similar(d)

In [ ]:
ID NAME Cred, Deb
1 Revenue   121 Tax.  21 TR 100
ID NAME Cred, Deb
2 Revenue   1210 Tax.  210 TR 1000
---
Normalize, then
ID NAME Cred, Deb
1 Revenue   0.8 Tax.  0.2 TR 1.0